<a href="https://colab.research.google.com/github/kyungmin0101/medical_ai/blob/main/0216/medicalAI_PCA%26LDA_kgh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import csv

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [12]:
data_path = '/content/drive/MyDrive/MedicalAI/1stSeries_07.05.21-20240214T041151Z-001/1stSeries_07.05.21'

from glob import glob
from os.path import isdir
df_list = []
fd_list = []

# read csv files
for x in glob(data_path+'/*/*.CSV'):
  if not 'REF' in x:

    f = open(x,'rt')
    reader = csv.reader(f)
    csv_content = []
    for raw in reader:
      csv_content.append(raw)
    f.close()

    df = pd.DataFrame(csv_content)
    df = df.drop(df.index[0:24])
    df_list.append(df)

    fd = x.split('/')
    name = fd[-2]
    fd_list.append(name)

  else:
    pass

# print results
print(df_list)
print(fd_list)

[              0           1
24    1475.0000  6.550E-007
25    1475.0198  9.910E-007
26    1475.0397  6.718E-007
27    1475.0595  7.304E-007
28    1475.0794  5.226E-007
...         ...         ...
6320  1599.9206  9.026E-007
6321  1599.9405  9.972E-007
6322  1599.9603  9.353E-007
6323  1599.9802  1.221E-006
6324  1600.0000  1.003E-006

[6301 rows x 2 columns],               0           1
24    1475.0000  6.206E-007
25    1475.0198  5.529E-007
26    1475.0397  5.166E-007
27    1475.0595  5.934E-007
28    1475.0794  1.919E-007
...         ...         ...
6320  1599.9206  9.918E-007
6321  1599.9405  1.373E-006
6322  1599.9603  9.101E-007
6323  1599.9802  6.177E-007
6324  1600.0000  1.189E-006

[6301 rows x 2 columns],               0           1
24    1475.0000  4.688E-007
25    1475.0198  1.006E-006
26    1475.0397  2.784E-007
27    1475.0595  5.537E-007
28    1475.0794  5.359E-007
...         ...         ...
6320  1599.9206  1.109E-006
6321  1599.9405  3.607E-007
6322  1599.9603  9.483E

In [17]:
### PDA & KNN
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score # train된 예측치(pred)와 실제 값을 비교하기 위한 function
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

#split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df_list, fd_list, test_size=0.1, random_state=42)
clf = KNeighborsClassifier(n_neighbors=2)

nsamples, nx, ny = np.array(x_train).shape
d2_x_train = np.array(x_train).reshape((nsamples,nx*ny))

nsamples, nx, ny = np.array(x_test).shape
d2_x_test = np.array(x_test).reshape((nsamples,nx*ny))

## dimensionality reduction(PCA)
pca = PCA(n_components = 2)
# training
pca.fit(d2_x_train,y_train)
com_x_train = pca.fit_transform(d2_x_train)

pca.fit(d2_x_test,y_test)
com_x_test = pca.fit_transform(d2_x_test)

# training
clf.fit(com_x_train,y_train)

# test
y_pred_pca = clf.predict(com_x_test) #clf.predict(X_test,y_test)는 불가능
print('pred_pca',y_pred_pca)
print('actual',y_test)


pred_pca ['1.41' '1.34' '1.34' '1.44' '1.3' '1.43' '1.37' '1.43' '1.41' '1.38'
 '1.43' '1.31' '1.35' '1.43' '1.45' '1.44' '1.39' '1.37' '1.44' '1.43'
 '1.3' '1.4' '1.41' '1.43']
actual ['1.43', '1.34', '1.35', '1.44', '1.31', '1.49', '1.39', '1.47', '1.41', '1.42', '1.43', '1.31', '1.33', '1.48', '1.44', '1.46', '1.4', '1.39', '1.5', '1.48', '1.3', '1.41', '1.49', '1.48']


In [18]:
# Accuracy for PDA&KNN
# accuracy_score(y_test, y_pred)
print('PCA&KNN Acc : {}'.format(accuracy_score(y_pred_pca,y_test)))
print('PCA&KNN f1 : {}'.format(f1_score(y_pred_pca,y_test,average='micro')))

PCA&KNN Acc : 0.25
PCA&KNN f1 : 0.25


In [13]:
### LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # train된 예측치(pred)와 실제 값을 비교하기 위한 function

#split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(df_list, fd_list, test_size=0.1, random_state=42)

nsamples, nx, ny = np.array(x_train).shape
d2_x_train = np.array(x_train).reshape((nsamples,nx*ny))
print(np.array(d2_x_train).shape)

nsamples, nx, ny = np.array(x_test).shape
d2_x_test = np.array(x_test).reshape((nsamples,nx*ny))

## LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)

# training
lda.fit(d2_x_train,y_train)
print("scale data shape x_train : {com_x_train.shape}")
print("scale data shape x_test : {com_x_tes.shape}")


# test
y_pred_lda =lda.predict(d2_x_test) #clf.predict(X_test,y_test)는 불가능
print('pred_lda : ',y_pred_lda)
print('actual : ',y_test)


(207, 12602)
scale data shape x_train : {com_x_train.shape}
scale data shape x_test : {com_x_tes.shape}
pred_lda :  ['1.43' '1.34' '1.35' '1.44' '1.31' '1.48' '1.39' '1.47' '1.41' '1.42'
 '1.43' '1.3' '1.33' '1.48' '1.44' '1.46' '1.4' '1.39' '1.5' '1.48' '1.3'
 '1.43' '1.49' '1.47']
actual :  ['1.43', '1.34', '1.35', '1.44', '1.31', '1.49', '1.39', '1.47', '1.41', '1.42', '1.43', '1.31', '1.33', '1.48', '1.44', '1.46', '1.4', '1.39', '1.5', '1.48', '1.3', '1.41', '1.49', '1.48']


In [14]:
# Accuracy for LDA
print('LDA Acc : {}'.format(accuracy_score(y_test,y_pred_lda)))

LDA Acc : 0.8333333333333334
